In [83]:
import pandas as pd
import numpy as np

In [84]:
trip = pd.read_csv('data/tripP')
weather = pd.read_csv('data/weatherP')
station = pd.read_csv('data/stationP')

/home/pabloernesto/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [85]:
trip = trip.merge(station, on='start_station_id', how='left')

In [86]:
trip = trip.merge(weather, on=['date', 'zip_code'], how='left')
#SOLUCION RATA
trip.fillna(0, inplace=True)
trip.drop(['date', 'zip_code'], axis=1, inplace=True)

In [87]:
trip = trip.drop(['holiday'], axis=1)

In [88]:
def sepTrainTest(df):
    train = df.loc[:df.shape[0] - 50000]
    test = df.loc[-50000:]
    return extractxy(train), extractxy(test)
    
def extractxy(df):
    y = df.duration
    del df['duration']
    return df, y

In [89]:
train = pd.read_csv('data/trip_train.csv', usecols=['id'])
train = train.merge(trip, on='id', how='left')
train = extractxy(train)

In [90]:
test = pd.read_csv('data/trip_test.csv', usecols=['id'])
test = test.merge(trip, how="left", on="id")
test = extractxy(test)

In [91]:
train[0].drop('date.1', axis=1, inplace=True)
test[0].drop('date.1', axis=1, inplace=True)

In [92]:
from sklearn import ensemble
#rfr = ensemble.RandomForestRegressor(n_jobs=-1)
rfr = ensemble.ExtraTreesRegressor(n_jobs=-1)

In [93]:
rfr.fit(train[0], train[1])

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
          verbose=0, warm_start=False)

In [94]:
from sklearn.metrics import mean_squared_error
mean_squared_error(rfr.predict(test[0]), test[1])

71141955.29977107